In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.tea

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!rm -rf spark-3.3.0-bin-hadoop3.tgz*

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.3.0-bin-hadoop3'

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession

In [ ]:
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
# Check Spark Session Information
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
text1 = open('first.txt','r')
text2 = open('second.txt','r')
text3 = open('third.txt','r')

In [ ]:
data=[(1,text1.read()),(2,text2.read()),(3,text3.read())]
data

[(1, 'i love dogs'),
 (2, 'i hate dogs and knitting'),
 (3, 'knitting is my hobby and my passion')]

In [ ]:
#data=[(1,'i love dogs'),(2,"i hate dogs and knitting"),(3,"knitting is my hobby and my passion")]
lines=sc.parallelize(data)
lines.collect()

[(1, 'i love dogs'),
 (2, 'i hate dogs and knitting'),
 (3, 'knitting is my hobby and my passion')]

In [ ]:
map1=lines.flatMap(lambda x: [((x[0],word),1) for word in x[1].split()])
map1.collect()

[((1, 'i'), 1),
 ((1, 'love'), 1),
 ((1, 'dogs'), 1),
 ((2, 'i'), 1),
 ((2, 'hate'), 1),
 ((2, 'dogs'), 1),
 ((2, 'and'), 1),
 ((2, 'knitting'), 1),
 ((3, 'knitting'), 1),
 ((3, 'is'), 1),
 ((3, 'my'), 1),
 ((3, 'hobby'), 1),
 ((3, 'and'), 1),
 ((3, 'my'), 1),
 ((3, 'passion'), 1)]

In [ ]:
reduce=map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((2, 'i'), 1),
 ((2, 'hate'), 1),
 ((2, 'dogs'), 1),
 ((3, 'knitting'), 1),
 ((3, 'my'), 2),
 ((3, 'hobby'), 1),
 ((3, 'and'), 1),
 ((1, 'i'), 1),
 ((1, 'love'), 1),
 ((1, 'dogs'), 1),
 ((2, 'and'), 1),
 ((2, 'knitting'), 1),
 ((3, 'is'), 1),
 ((3, 'passion'), 1)]

In [ ]:
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

[('i', (2, 1)),
 ('hate', (2, 1)),
 ('dogs', (2, 1)),
 ('knitting', (3, 1)),
 ('my', (3, 2)),
 ('hobby', (3, 1)),
 ('and', (3, 1)),
 ('i', (1, 1)),
 ('love', (1, 1)),
 ('dogs', (1, 1)),
 ('and', (2, 1)),
 ('knitting', (2, 1)),
 ('is', (3, 1)),
 ('passion', (3, 1))]

In [ ]:
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()

[('i', (2, 1, 1)),
 ('hate', (2, 1, 1)),
 ('dogs', (2, 1, 1)),
 ('knitting', (3, 1, 1)),
 ('my', (3, 2, 1)),
 ('hobby', (3, 1, 1)),
 ('and', (3, 1, 1)),
 ('i', (1, 1, 1)),
 ('love', (1, 1, 1)),
 ('dogs', (1, 1, 1)),
 ('and', (2, 1, 1)),
 ('knitting', (2, 1, 1)),
 ('is', (3, 1, 1)),
 ('passion', (3, 1, 1))]

In [ ]:
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()

[('i', 1),
 ('hate', 1),
 ('dogs', 1),
 ('knitting', 1),
 ('my', 1),
 ('hobby', 1),
 ('and', 1),
 ('i', 1),
 ('love', 1),
 ('dogs', 1),
 ('and', 1),
 ('knitting', 1),
 ('is', 1),
 ('passion', 1)]

In [ ]:
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()

[('i', 2),
 ('hate', 1),
 ('dogs', 2),
 ('love', 1),
 ('is', 1),
 ('passion', 1),
 ('knitting', 2),
 ('my', 1),
 ('hobby', 1),
 ('and', 2)]

In [ ]:
import math
from pyspark.sql.functions import *
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()

[('i', 0.17609125905568124),
 ('hate', 0.47712125471966244),
 ('dogs', 0.17609125905568124),
 ('love', 0.47712125471966244),
 ('is', 0.47712125471966244),
 ('passion', 0.47712125471966244),
 ('knitting', 0.17609125905568124),
 ('my', 0.47712125471966244),
 ('hobby', 0.47712125471966244),
 ('and', 0.17609125905568124)]

In [ ]:
rdd=tf.join(idf)
rdd.collect()

[('i', ((2, 1), 0.17609125905568124)),
 ('i', ((1, 1), 0.17609125905568124)),
 ('dogs', ((2, 1), 0.17609125905568124)),
 ('dogs', ((1, 1), 0.17609125905568124)),
 ('passion', ((3, 1), 0.47712125471966244)),
 ('knitting', ((3, 1), 0.17609125905568124)),
 ('knitting', ((2, 1), 0.17609125905568124)),
 ('hobby', ((3, 1), 0.47712125471966244)),
 ('and', ((3, 1), 0.17609125905568124)),
 ('and', ((2, 1), 0.17609125905568124)),
 ('hate', ((2, 1), 0.47712125471966244)),
 ('love', ((1, 1), 0.47712125471966244)),
 ('is', ((3, 1), 0.47712125471966244)),
 ('my', ((3, 2), 0.47712125471966244))]

In [ ]:
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()

[(1, ('i', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('dogs', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('love', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('i', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('dogs', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('knitting', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('and', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('hate', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('passion', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('knitting', 1, 0.17609125905568124, 0.17609125905568124)),
 (3, ('hobby', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('and', 1, 0.17609125905568124, 0.17609125905568124)),
 (3, ('is', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('my', 2, 0.47712125471966244, 0.9542425094393249))]

In [ ]:
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
rdd.toDF(["DocumentId","Word","TF","IDF","TF-IDF"]).show()

+----------+--------+---+-------------------+-------------------+
|DocumentId|    Word| TF|                IDF|             TF-IDF|
+----------+--------+---+-------------------+-------------------+
|         1|       i|  1|0.17609125905568124|0.17609125905568124|
|         1|    dogs|  1|0.17609125905568124|0.17609125905568124|
|         1|    love|  1|0.47712125471966244|0.47712125471966244|
|         2|       i|  1|0.17609125905568124|0.17609125905568124|
|         2|    dogs|  1|0.17609125905568124|0.17609125905568124|
|         2|knitting|  1|0.17609125905568124|0.17609125905568124|
|         2|     and|  1|0.17609125905568124|0.17609125905568124|
|         2|    hate|  1|0.47712125471966244|0.47712125471966244|
|         3| passion|  1|0.47712125471966244|0.47712125471966244|
|         3|knitting|  1|0.17609125905568124|0.17609125905568124|
|         3|   hobby|  1|0.47712125471966244|0.47712125471966244|
|         3|     and|  1|0.17609125905568124|0.17609125905568124|
|         